In [ ]:
# -*- coding: utf-8 -*-
"""
RUN ALL ANALYSES: Reproducing All Tables and Figures
This notebook executes the full pipeline of the study:
1. Basic academic inequality (Gini, top 10%, h-index)
2. Temporal trends (2015–2024)
3. Global North vs. Global South statistical comparison

All outputs are saved to:
- Figures: ../output/figures/
- Tables:  ../output/tables/
"""

# =============================================================================
# CELL 1: SETUP AND DATA LOADING (SHARED ACROSS ALL ANALYSES)
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import warnings
warnings.filterwarnings("ignore")

# Set relative paths (relative to notebooks/)
input_file = "../data/processed/processed_publications_for_mertonian_analysis.csv"
output_figures = "../output/figures"
output_tables = "../output/tables"

# Create output directories
os.makedirs(output_figures, exist_ok=True)
os.makedirs(output_tables, exist_ok=True)

# Load and inspect data
try:
    data = pd.read_csv(input_file)
    print("✅ Dataset loaded successfully.")
except FileNotFoundError:
    raise FileNotFoundError(f"Input file not found: {input_file}")

print(f"📊 Dataset: {data.shape[0]} records × {data.shape[1]} variables")
print("\n📋 Columns:", data.columns.tolist())
print()

# Helper: find column case-insensitively
def find_column(data, candidates):
    for name in candidates:
        if name.lower() in [col.lower() for col in data.columns]:
            return [col for col in data.columns if col.lower() == name.lower()][0]
    return None

# Identify key columns
author_col = find_column(data, ['Penulis_Utama', 'First_Author', 'Author', 'Authors'])
citation_col = find_column(data, ['Jumlah_Sitasi', 'Citations', 'Cited by', 'cited_by'])
year_col = find_column(data, ['Tahun', 'Year', 'PY'])
region_col = find_column(data, ['Region', 'Country_Group', 'Global_North_South', 'Affiliation_Region'])

# Validate essential columns
if not author_col or not citation_col:
    raise KeyError("Required columns (author, citations) not found.")

# Standardize
data_clean = data.rename(columns={
    author_col: 'Primary_Author',
    citation_col: 'Citation_Count'
})

# Clean citations
data_clean['Citation_Count'] = pd.to_numeric(data_clean['Citation_Count'], errors='coerce')
data_clean = data_clean.dropna(subset=['Primary_Author', 'Citation_Count'])
data_clean = data_clean[data_clean['Citation_Count'] >= 0]

# Clean author names
data_clean['Primary_Author'] = data_clean['Primary_Author'].astype(str).str.strip()
data_clean = data_clean[data_clean['Primary_Author'] != '']

print(f"🧹 Cleaned dataset: {len(data_clean)} publications")
print("="*70)

# =============================================================================
# CELL 2: BASIC INEQUALITY ANALYSIS (Gini, Top 10%, h-index)
# =============================================================================
print("\n🔬 ANALYSIS 1: BASIC ACADEMIC INEQUALITY METRICS\n" + "-"*50)

def compute_gini(x):
    x = np.array(x)
    if len(x) == 0 or x.sum() == 0:
        return 0.0
    x = x[x > 0]
    x = np.sort(x)
    n = len(x)
    index = np.arange(1, n + 1)
    return (np.sum((2 * index - n - 1) * x)) / (n * np.sum(x))

# Aggregate by author
citations_per_author = data_clean.groupby('Primary_Author')['Citation_Count'].sum().sort_values(ascending=False).values
gini_coef = compute_gini(citations_per_author)

total_citations = citations_per_author.sum()
total_authors = len(citations_per_author)
n_top_10 = max(1, int(0.1 * total_authors))
pct_top_10 = (citations_per_author[:n_top_10].sum() / total_citations) * 100
h_index = np.sum(citations_per_author >= np.arange(1, len(citations_per_author) + 1))

# Display and save
basic_results = pd.DataFrame({
    'Metric': [
        'Gini Coefficient',
        '% Citations by Top 10%',
        'Estimated Global h-index',
        'Total Unique Authors',
        'Size of Top 10% Group'
    ],
    'Value': [
        f"{gini_coef:.3f}",
        f"{pct_top_10:.1f}%",
        h_index,
        total_authors,
        n_top_10
    ]
})
print(basic_results.to_string(index=False))

basic_results.to_csv(os.path.join(output_tables, "inequality_summary.csv"), index=False)

# Plot basic visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Academic Influence Inequality (Basic Metrics)', fontsize=16, fontweight='bold')

# Lorenz Curve
cumulative_share = np.cumsum(citations_per_author) / total_citations
pop_cum = np.arange(1, len(cumulative_share)+1) / len(cumulative_share)
axes[0,0].plot([0,1], [0,1], '--', color='gray')
axes[0,0].plot(pop_cum, cumulative_share, color='red')
axes[0,0].set_title(f'Lorenz Curve (Gini = {gini_coef:.3f})')
axes[0,0].set_xlabel('Cumulative Authors (%)')
axes[0,0].set_ylabel('Cumulative Citations (%)')

# Top 10% pie
axes[0,1].pie([pct_top_10, 100-pct_top_10], labels=['Top 10%', 'Other 90%'], autopct='%1.1f%%', colors=['#ff9999','#66b3ff'])
axes[0,1].set_title('% Citations by Top 10% Authors')

# Top 20 authors by pubs
pubs = data_clean['Primary_Author'].value_counts().head(20)
pubs.plot(kind='bar', ax=axes[1,0], color='skyblue')
axes[1,0].set_title('Publications per Author (Top 20)')
axes[1,0].tick_params(axis='x', rotation=45)

# Top 20 by citations
top20_cit = data_clean.groupby('Primary_Author')['Citation_Count'].sum().sort_values(ascending=False).head(20)
top20_cit.plot(kind='bar', ax=axes[1,1], color='salmon')
axes[1,1].set_title('Total Citations per Author (Top 20)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(os.path.join(output_figures, "academic_inequality_analysis.png"), dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Basic analysis outputs saved to:\n  - {output_tables}/inequality_summary.csv\n  - {output_figures}/academic_inequality_analysis.png")
print("="*70)

# =============================================================================
# CELL 3: TEMPORAL TRENDS (2015–2024)
# =============================================================================
print("\n🔬 ANALYSIS 2: TEMPORAL TRENDS (2015–2024)\n" + "-"*50)

# Add year if available
if year_col:
    data_clean['Publication_Year'] = pd.to_numeric(data_clean[year_col], errors='coerce')
    data_clean = data_clean.dropna(subset=['Publication_Year'])
    data_clean = data_clean[(data_clean['Publication_Year'] >= 2015) & (data_clean['Publication_Year'] <= 2024)]
    
    period_1 = data_clean[(data_clean['Publication_Year'] >= 2015) & (data_clean['Publication_Year'] <= 2019)].copy()
    period_2 = data_clean[(data_clean['Publication_Year'] >= 2020) & (data_clean['Publication_Year'] <= 2024)].copy()
    
    print(f"📅 Period 1 (2015–2019): {len(period_1)} publications")
    print(f"📅 Period 2 (2020–2024): {len(period_2)} publications")
    
    def compute_metrics(df):
        if len(df) == 0:
            return None
        cit = df.groupby('Primary_Author')['Citation_Count'].sum().sort_values(ascending=False).values
        gini = compute_gini(cit)
        total = cit.sum()
        authors = len(cit)
        n_top = max(1, int(0.1 * authors))
        pct = (cit[:n_top].sum() / total) * 100 if total > 0 else 0
        h_idx = np.sum(cit >= np.arange(1, len(cit)+1))
        return {'Gini': round(gini,3), 'Top10Pct': f"{pct:.1f}%", 'h_index': h_idx, 'Authors': authors}
    
    m1 = compute_metrics(period_1)
    m2 = compute_metrics(period_2)
    
    if m1 and m2:
        temporal_table = pd.DataFrame({
            'Metric': ['Gini Coefficient', 'Top 10% Citation Share', 'Estimated h-index', 'Total Unique Authors'],
            '2015–2019': [m1['Gini'], m1['Top10Pct'], m1['h_index'], m1['Authors']],
            '2020–2024': [m2['Gini'], m2['Top10Pct'], m2['h_index'], m2['Authors']]
        })
        print(temporal_table.to_string(index=False))
        temporal_table.to_csv(os.path.join(output_tables, "temporal_inequality_trends.csv"), index=False)
        
        # Plot trends
        fig, axes = plt.subplots(2, 2, figsize=(14,10))
        fig.suptitle('Temporal Trends in Academic Inequality (2015–2024)', fontsize=16, fontweight='bold')
        colors = ['#4E79A7', '#F28E2B']
        
        metrics = [
            ([m1['Gini'], m2['Gini']], 'Gini Coefficient', 'Gini Value'),
            ([float(m1['Top10Pct'].strip('%')), float(m2['Top10Pct'].strip('%'))], 'Top 10% Citation Share', 'Percentage (%)'),
            ([m1['h_index'], m2['h_index']], 'Estimated Global h-index', 'h-index'),
            ([m1['Authors'], m2['Authors']], 'Total Unique Authors', 'Count')
        ]
        
        for i, (vals, title, ylabel) in enumerate(metrics):
            ax = axes[i//2, i%2]
            bars = ax.bar(['2015–2019', '2020–2024'], vals, color=colors, edgecolor='black')
            ax.set_title(title, fontweight='bold')
            ax.set_ylabel(ylabel)
            for bar, v in zip(bars, vals):
                ax.text(bar.get_x()+bar.get_width()/2, bar.get_height()+ (0.02 if i==0 else (1 if i==1 else (2 if i==2 else 10))), 
                        f"{v:.3f}" if i==0 else (f"{v:.1f}%" if i==1 else str(int(v))), 
                        ha='center', va='bottom', fontsize=10)
            ax.spines[['top','right']].set_visible(False)
        
        plt.tight_layout(rect=[0,0,1,0.95])
        plt.savefig(os.path.join(output_figures, "academic_inequality_trends.png"), dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"✅ Temporal analysis outputs saved to:\n  - {output_tables}/temporal_inequality_trends.csv\n  - {output_figures}/academic_inequality_trends.png")
    else:
        print("⚠️ Not enough data for temporal analysis.")
else:
    print("⚠️ Year column not found. Skipping temporal analysis.")

print("="*70)

# =============================================================================
# CELL 4: GLOBAL NORTH vs. GLOBAL SOUTH COMPARISON
# =============================================================================
print("\n🔬 ANALYSIS 3: GLOBAL NORTH vs. GLOBAL SOUTH\n" + "-"*50)

if region_col:
    data_clean['Region_Group'] = data_clean[region_col].astype(str).str.strip().str.title()
    
    # Standardize regions
    north_labels = {'Global North', 'North', 'Developed', 'High Income'}
    south_labels = {'Global South', 'South', 'Developing', 'Low Income', 'Middle Income'}
    
    def classify_region(label):
        if label in north_labels:
            return 'Global North'
        elif label in south_labels:
            return 'Global South'
        else:
            return label  # keep as-is if unknown
    
    data_clean['Region_Std'] = data_clean['Region_Group'].apply(classify_region)
    
    if 'Global North' in data_clean['Region_Std'].values and 'Global South' in data_clean['Region_Std'].values:
        filtered = data_clean[data_clean['Region_Std'].isin(['Global North', 'Global South'])]
        gn = filtered[filtered['Region_Std'] == 'Global North']['Citation_Count']
        gs = filtered[filtered['Region_Std'] == 'Global South']['Citation_Count']
        
        print(f"🌍 Global North: {len(gn)} publications")
        print(f"🌍 Global South: {len(gs)} publications")
        
        # Statistical test
        u_stat, p_val = stats.mannwhitneyu(gn, gs, alternative='greater')
        def cliffs_delta(x, y):
            n_x, n_y = len(x), len(y)
            delta = sum(1 if xi > yj else (-1 if xi < yj else 0) for xi in x for yj in y)
            return delta / (n_x * n_y) if n_x * n_y > 0 else np.nan
        cliff_d = cliffs_delta(gn.values, gs.values)
        
        # Interpret effect
        d_abs = abs(cliff_d)
        effect = "Negligible" if d_abs < 0.147 else "Small" if d_abs < 0.33 else "Medium" if d_abs < 0.474 else "Large"
        
        # Results table
        stats_results = pd.DataFrame({
            'Metric': [
                'Mann-Whitney U',
                'p-value',
                "Cliff's Delta",
                'Effect Size',
                'Median Citations (GN)',
                'Median Citations (GS)'
            ],
            'Value': [
                round(u_stat, 2),
                f"{p_val:.2e}" if p_val < 0.001 else f"{p_val:.4f}",
                round(cliff_d, 3),
                effect,
                round(gn.median(), 2),
                round(gs.median(), 2)
            ]
        })
        print(stats_results.to_string(index=False))
        stats_results.to_csv(os.path.join(output_tables, "north_south_citation_inequality_stats.csv"), index=False)
        
        # Plot
        plt.figure(figsize=(10,6))
        sns.boxplot(data=filtered, x='Region_Std', y='Citation_Count', order=['Global North','Global South'], 
                    palette=['#4E79A7','#F28E2B'])
        plt.yscale('log')
        plt.title('Citation Distribution: Global North vs. Global South', fontweight='bold')
        plt.xlabel('Region Group')
        plt.ylabel('Citations (log scale)')
        for i, group in enumerate(['Global North','Global South']):
            n = len(filtered[filtered['Region_Std'] == group])
            plt.text(i, plt.ylim()[0]*1.2, f'n = {n}', ha='center', fontweight='bold')
        plt.tight_layout()
        plt.savefig(os.path.join(output_figures, "north_south_citation_distribution.png"), dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"✅ North-South analysis outputs saved to:\n  - {output_tables}/north_south_citation_inequality_stats.csv\n  - {output_figures}/north_south_citation_distribution.png")
    else:
        print("⚠️ Could not identify both 'Global North' and 'Global South' groups.")
else:
    print("⚠️ Region column not found. Skipping North-South analysis.")

print("\n" + "="*70)
print("🎉 ALL ANALYSES COMPLETED SUCCESSFULLY!")
print(f"📁 All outputs are saved in:\n  - {output_figures}/\n  - {output_tables}/")
print("💡 To reproduce: Run all cells in this notebook.")